# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [1]:
# Your code here
import pandas as pd
import sqlite3

conn = sqlite3.connect('data.sqlite')

## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [3]:
# Your code here
pd.read_sql("""
            SELECT firstName, lastName
            FROM employees JOIN  offices USING(officeCode)
""", conn)

,firstName,lastName
0,Diane,Murphy
1,Mary,Patterson
2,Jeff,Firrelli
3,William,Patterson
4,Gerard,Bondur
5,Anthony,Bow
6,Leslie,Jennings
7,Leslie,Thompson
8,Julie,Firrelli
9,Steve,Patterson


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [5]:
# Your code here
pd.read_sql("""
            SELECT officeCode, city, COUNT() AS numberOfEmployees
            FROM employees JOIN  offices USING(officeCode)
            GROUP BY officeCode
""", conn)

,officeCode,city,numberOfEmployees
0,1,San Francisco,6
1,2,Boston,2
2,3,NYC,2
3,4,Paris,5
4,5,Tokyo,2
5,6,Sydney,4
6,7,London,2


## Write 3 questions of your own and answer them

In [ ]:
# Answers will vary

# Example question: 
"""
How many customers are there per office?
"""

In [9]:
"""
Question 1:
How many orders have the 10 most active customers placed?
"""

# Your code here
pd.read_sql("""
            SELECT customerName, COUNT() AS numberOfOrders
            FROM customers JOIN  orders USING(customerNumber)
            GROUP BY customerNumber
            ORDER BY numberOfOrders DESC
            LIMIT 10
""", conn)

,customerName,numberOfOrders
0,Euro+ Shopping Channel,26
1,Mini Gifts Distributors Ltd.,17
2,Reims Collectables,5
3,"Down Under Souveniers, Inc",5
4,"Dragon Souveniers, Ltd.",5
5,Danish Wholesale Imports,5
6,"Australian Collectors, Co.",5
7,Kelly's Gift Shop,4
8,The Sharp Gifts Warehouse,4
9,"Tokyo Collectables, Ltd",4


In [15]:
"""
Question 2
Which product lines have sold more than 10,000 units?
"""

# Your code here
pd.read_sql("""
            SELECT productLine, SUM(quantityOrdered) AS totalUnitsSold
            FROM (productLines AS pl JOIN products AS p USING(productLine)) JOIN orderDetails AS od USING(productCode)
            GROUP BY productLine
            HAVING totalUnitsSold > 10000

""", conn)

,productLine,totalUnitsSold
0,Classic Cars,35582
1,Motorcycles,12778
2,Planes,11872
3,Trucks and Buses,11001
4,Vintage Cars,22933


In [30]:
"""
Question 3:
Which are the 5 greatest single payments made by an american customer?
"""

# Your code here
pd.read_sql("""
            SELECT c.customerName, c.country, p.amount
            FROM customers AS c JOIN payments AS p USING(customerNumber)
            WHERE c.country = 'USA'
            ORDER BY p.amount DESC
            LIMIT 5
""", conn)

,customerName,country,amount
0,Mini Gifts Distributors Ltd.,USA,111654.40
1,Mini Gifts Distributors Ltd.,USA,101244.59
2,Corporate Gift Ideas Co.,USA,85559.12
3,Mini Gifts Distributors Ltd.,USA,85410.87
4,Mini Gifts Distributors Ltd.,USA,83598.04


## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [45]:
# Your code here
pd.read_sql("""
            SELECT DISTINCT e.employeeNumber, e.firstName, e.lastName, p.productName
            FROM products AS p
                JOIN orderDetails AS od USING (productCode)
                JOIN orders AS o USING (orderNumber)
                JOIN customers AS c USING (customerNumber)
                JOIN employees AS e ON c.salesRepEmployeeNumber = e.employeeNumber
            ORDER BY e.firstName
""", conn)

,employeeNumber,firstName,lastName,productName
0,1611,Andy,Fixter,1969 Harley Davidson Ultimate Chopper
1,1611,Andy,Fixter,1952 Alpine Renault 1300
2,1611,Andy,Fixter,1996 Moto Guzzi 1100i
3,1611,Andy,Fixter,2003 Harley-Davidson Eagle Drag Bike
4,1611,Andy,Fixter,1962 LanciaA Delta 16V
...,...,...,...,...
1363,1216,Steve,Patterson,The Titanic
1364,1216,Steve,Patterson,The Queen Mary
1365,1216,Steve,Patterson,American Airlines: MD-11S
1366,1216,Steve,Patterson,Boeing X-32A JSF


## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [49]:
# Your code here
pd.read_sql("""
            SELECT e.employeeNumber, e.firstName, e.lastName, SUM(od.quantityOrdered) AS productsSold
            FROM products AS p
                JOIN orderDetails AS od USING (productCode)
                JOIN orders AS o USING (orderNumber)
                JOIN customers AS c USING (customerNumber)
                JOIN employees AS e ON c.salesRepEmployeeNumber = e.employeeNumber
            GROUP BY e.employeeNumber
""", conn)

,employeeNumber,firstName,lastName,productsSold
0,1165,Leslie,Jennings,11854
1,1166,Leslie,Thompson,4056
2,1188,Julie,Firrelli,4227
3,1216,Steve,Patterson,5561
4,1286,Foon Yue,Tseng,5016
5,1323,George,Vanauf,7423
6,1337,Loui,Bondur,6186
7,1370,Gerard,Hernandez,14231
8,1401,Pamela,Castillo,9290
9,1501,Larry,Bott,8205


## Level Up 3: Display the names employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [52]:
# Your code here
pd.read_sql("""
            SELECT e.employeeNumber, e.firstName, e.lastName, COUNT(DISTINCT od.productCode) AS productsSold
            FROM products AS p
                JOIN orderDetails AS od USING (productCode)
                JOIN orders AS o USING (orderNumber)
                JOIN customers AS c USING (customerNumber)
                JOIN employees AS e ON c.salesRepEmployeeNumber = e.employeeNumber
            GROUP BY e.employeeNumber
""", conn)

,employeeNumber,firstName,lastName,productsSold
0,1165,Leslie,Jennings,107
1,1166,Leslie,Thompson,73
2,1188,Julie,Firrelli,80
3,1216,Steve,Patterson,95
4,1286,Foon Yue,Tseng,74
5,1323,George,Vanauf,96
6,1337,Loui,Bondur,101
7,1370,Gerard,Hernandez,109
8,1401,Pamela,Castillo,100
9,1501,Larry,Bott,97


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!